In [1]:
import nltk
nltk.download('wordnet')
nltk.download('punkt') # Tokenizer model
nltk.download()


from nltk import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()


print(lemmatizer.lemmatize('are'))
print(lemmatizer.lemmatize('carss'))
print(lemmatizer.lemmatize("cats"))
print(lemmatizer.lemmatize("cacti"))
print(lemmatizer.lemmatize("geese"))
print(lemmatizer.lemmatize("rocks"))

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
are
car
cat
cactus
goose
rock


In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Using TensorFlow backend.


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']
Training data created

Epoch 1/200
4

47/47 [==============================] - 0s 621us/step - loss: 0.0928 - accuracy: 0.9787
Epoch 70/200
47/47 [==============================] - 0s 704us/step - loss: 0.0399 - accuracy: 1.0000
Epoch 71/200
47/47 [==============================] - 0s 572us/step - loss: 0.1087 - accuracy: 0.9362
Epoch 72/200
47/47 [==============================] - 0s 489us/step - loss: 0.0406 - accuracy: 1.0000
Epoch 73/200
47/47 [==============================] - 0s 503us/step - loss: 0.0560 - accuracy: 1.0000
Epoch 74/200
47/47 [==============================] - 0s 711us/step - loss: 0.0618 - accuracy: 0.9787
Epoch 75/200
47/47 [==============================] - 0s 622us/step - loss: 0.0948 - accuracy: 0.9362
Epoch 76/200
47/47 [==============================] - 0s 624us/step - loss: 0.0784 - accuracy: 0.9574
Epoch 77/200
47/47 [==============================] - 0s 683us/step - loss: 0.0432 - accuracy: 0.9787
Epoch 78/200
47/47 [==============================] - 0s 601us/step - loss: 0.0560 - accuracy: 

47/47 [==============================] - 0s 621us/step - loss: 0.0867 - accuracy: 0.9787
Epoch 149/200
47/47 [==============================] - 0s 568us/step - loss: 0.0125 - accuracy: 1.0000
Epoch 150/200
47/47 [==============================] - 0s 687us/step - loss: 0.0150 - accuracy: 1.0000
Epoch 151/200
47/47 [==============================] - 0s 814us/step - loss: 0.0502 - accuracy: 0.9787
Epoch 152/200
47/47 [==============================] - 0s 802us/step - loss: 0.0487 - accuracy: 1.0000
Epoch 153/200
47/47 [==============================] - 0s 701us/step - loss: 0.0231 - accuracy: 1.0000
Epoch 154/200
47/47 [==============================] - 0s 688us/step - loss: 0.0540 - accuracy: 0.9787
Epoch 155/200
47/47 [==============================] - 0s 668us/step - loss: 0.0547 - accuracy: 0.9787
Epoch 156/200
47/47 [==============================] - 0s 678us/step - loss: 0.0299 - accuracy: 1.0000
Epoch 157/200
47/47 [==============================] - 0s 710us/step - loss: 0.0179 - a